In [1]:
%run stdPackages.ipynb

*Test database:*

In [2]:
from pyDbs import read, readSets, broadcast
kwargs = {'variables': ['Fundamentals', 'Load', 'Generators_Other'], 
        'variable2D': ['Generators_FuelMix','HourlyVariation'],
        'scalars': ['MWP'],
        'maps': ['Generators_Categories']}
db = read.dbFromWB(os.path.join(d['data'],'mBasicInt.xlsx'), kwargs)
db.updateAlias(alias=[('h','h_alias')])
readSets(db)

KeyError: 'Worksheet MWP does not exist.'

# lpCompiler, standard version 

This version of the ```lpCompiler``` works a lot like the old version, with the exception that the final collection of arguments is made sparse. This means that the compiler - in broad terms - consists of the following steps:
1. Add arguments for the five building blocks (```c, l, u, beq, bub, Aeq, Aub ```). All coefficients should be added with as ```pd.Series``` defined over appropriate ```pd.MultiIndex```. Relevant coefficients are stored in ```self.parameters``` - a dictionary with keys ```c, l, u, beq, bub, Aeq, Aub```.
2. Infer global domains from the building blocks and store at ```self.gIndex```. The global domains are stored as flattened multiindices with forced level names (elaboration below).
3. Broadcast coefficients to full domains (and sort). Relevant coefficients are stored in ```self.broadcasted``` - a dictionary with keys ```c, l, u, beq, bub, Aeq, Aub```. An option ```sparse = True``` makes the broadcasted series sparse (default is False).
4. The method ```self.lp_args``` finally stacks all relevant building blocks and returns a dictionary to be passed to the ```scipy.optimize.lp``` solver.

In [ ]:
_blocks = ('c','l','u','b_eq','b_ub','A_eq','A_ub')
_stdLinProg = ('c', 'A_ub','b_ub','A_eq','b_eq','bounds')
self = lpBlock()

## 1. Adding arguments

We specify three different methods for adding arguments: 
1. Add specific parameter to one of the five major blocks (```c,l,u,beq,bub,Aeq,Aub```). We specify separate functions for the different 5 blocks. 
2. Build entire parameter blocks from collection of arguments (faster). Speed up initialization, but less flexible.
3. Add constraint (relevant parts of ```b, A```) together.

### 1.1. Add specific parameter to ```self.parameters```

#### For ```c,l,u``` types: 

Should be added as pd.Series with appropriate values, indices, and name of series (corresponding to variable name). To distinguish between adding ```c,l,u``` type of arguments, we specify a separate add function for each of them (e.g. ```add_c```). More specifically:
* ```name```: The name input is required as it used to add/adjust/subtract coefficient blocks after initial compilation.
* ```value = None```: 
    * If the input is a scalar: The scalar is added with variable name ```varName```.
    * If the input is a pd.Series: The name of the series identifies the variable name. If ```None``` we default to the ```name```.
    * If the input is a list/tuple: The various components are summed/max/min if the block type is ```c,l,u``` respectively.
* ```varName = None```: If the input is a scalar, this specifies the name of this scalar.

*Note that this method of adding components implicitly assumes that each component only deals with one variable at a time.*

*NB: We should add a method that does not merge by adding and broadcasting, but instead simply stacks the arguments collected in some iterator.* 

In [ ]:
self.globalDomains['eq constr'] = db['h_alias'] # set global domains for the constraint to the hourly index
self.globalDomains['Generation'] = pd.MultiIndex.from_product([db['h'], db['id']])
self.globalDomains['HourlyDemand'] = db['h']

**Tests:**

In [ ]:
self.add_c('test c', value = adjMultiIndex.bc(db['OtherMC'] , db['h']), varName = 'Generation')
self.add_c('test sum c', value = [self.parameters['c'][('Generation','test c')]]*2, varName = 'Generation')
self.add_c('demand test', value = -10, varName = 'HourlyDemand')
self.add_u('test u', value = adjMultiIndex.bc(db['GeneratingCapacity'], db['h']), varName = 'Generation')
self.add_u('demand u', value = 10, varName = 'HourlyDemand')

#### For ```eq, ub``` types:

Equations and upper bound constraints consists of a vector of constants (```b```) and a matrix that relates to the variables (``` A ```). 

* Adding coefficients on constraint vectors (```beq,bub```): Similar way to ```c,l,u``` with the difference that names relative to the relevant constraint.

* Adding coefficients on constraint matrices (```Aeq,Aub```): Requires combination of name, constraint, and relevant variable. 
    * ```name```: The name is required as it is used to add/adjust/subtract coefficient blocks after initial compilation.
    * ```value=None```:
        * If scalar: 
    * ```varName=None```:  Name of variable relevant coefficients.
    * ```constrName=None```: Name of constraint.

In [ ]:
self.add_b_eq(value = None, constrName = 'eq constr')
self.add_A_eq(value = appIndexWithCopySeries(pd.Series(1, index = self.globalDomains['Generation']), 'h','h_alias'), constrName = 'eq constr', varName = 'Generation')
self.add_A_eq(value =appIndexWithCopySeries(pd.Series(-1, index = self.globalDomains['HourlyDemand']), 'h','h_alias'), constrName = 'eq constr', varName = 'HourlyDemand')

### 1.2. TO DO
### 1.3. TO DO

## 2. Compile - create 1d/2d multiindices

In [ ]:
self.compileParameters()

## 3. Settings from compiled

In [ ]:
self.settingsFromCompiled()

## 4. Infer global domains

Infer the relevant domains for variables based on entries in coefficient matrices:

In [ ]:
self.inferGlobalDomains()

## 5. Compile

In [ ]:
self.getDenseArgs()

Collect arguments:

## 6. Return lp arguments:

In [ ]:
self.lp_args

## 7. Using the ```__call__``` method

Initialize again and add model specifications:

In [ ]:
self = lpBlock()
self.globalDomains['eq constr'] = db['h_alias'] # set global domains for the constraint to the hourly index
self.globalDomains['Generation'] = pd.MultiIndex.from_product([db['h'], db['id']])
self.globalDomains['HourlyDemand'] = db['h']
self.add_c('test c', value = adjMultiIndex.bc(db['OtherMC'] , db['h']), varName = 'Generation')
self.add_c('test sum c', value = [self.parameters['c'][('Generation','test c')]]*2, varName = 'Generation')
self.add_c('demand test', value = -10, varName = 'HourlyDemand')
self.add_u('test u', value = adjMultiIndex.bc(db['GeneratingCapacity'], db['h']), varName = 'Generation')
self.add_u('demand u', value = 10, varName = 'HourlyDemand')
self.add_b_eq(value = None, constrName = 'eq constr')
self.add_A_eq(value = appIndexWithCopySeries(pd.Series(1, index = self.globalDomains['Generation']), 'h','h_alias'), constrName = 'eq constr', varName = 'Generation')
self.add_A_eq(value =appIndexWithCopySeries(pd.Series(-1, index = self.globalDomains['HourlyDemand']), 'h','h_alias'), constrName = 'eq constr', varName = 'HourlyDemand')

Use ```__call__```  to compile and everything:

In [ ]:
args = self()